Based on https://bambinos.github.io/bambi/notebooks/survival_model.html

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy

from lifelines import KaplanMeierFitter

import bambi as bmb

In [ ]:
url = (
    "https://raw.githubusercontent.com/rmcelreath/rethinking/master/data/AustinCats.csv"
)
cats_df = pd.read_csv(url, sep=";")

In [ ]:
plt.figure(figsize=(7, 3))
plt.hist(cats_df["days_to_event"], bins=250, label="Uncensored data")
plt.xlim(0, 186)  # limit to 6 months for visibility
plt.title("Days Until Adoption")
plt.ylabel("Count")
plt.xlabel("Days")
plt.legend();

In [ ]:
km = KaplanMeierFitter()
km_adoptions = km.fit(
    cats_df["days_to_event"],
    cats_df["out_event"].apply(lambda x: 1 if x == "Adoption" else 0),
)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3))
km_adoptions.plot(label="Kaplan-Meier Estimator", ax=ax)
ax.set_ylabel("Probability of Adoption")
ax.set_xlabel("Days")
ax.set_xlim(0, 365)
ax.grid(True)
ax.set_title("Cat Adoption Survival Curve");

In [ ]:
cats = cats_df.copy()
cats["adopt"] = np.where(cats["out_event"] == "Adoption", "right", "none")
cats["color_id"] = np.where(cats["color"] == "Black", 1, 0)
cats = cats[["days_to_event", "adopt", "color_id"]]

In [ ]:
model_1 = bmb.Model(
    "censored(days_to_event / 31, adopt) ~ 1",
    data=cats,
    family="exponential",
    link="log",
)
model_1.build()
model_1.graph()

The above cell raises `TypeError: unsupported operand type(s) for -: 'int' and 'method'`, skipping the related parts

In [ ]:
cat_model = bmb.Model(
    "censored(days_to_event / 31, adopt) ~ 0 + color_id",
    data=cats,
    center_predictors=False,
    priors={"color_id": bmb.Prior("Normal", mu=0, sigma=1)},
    categorical=["color_id"],
    family="exponential",
    link="log",
)
cat_model.build()
cat_model.graph()

The above cell raises `TypeError: unsupported operand type(s) for -: 'int' and 'method'`, skipping the related parts